# Prostate segmentation on IDC collection -- Qin-Prostate-Repeatablity
*   Dataset : [Qin-Prostate-Repeatability](https://portal.imaging.datacommons.cancer.gov/explore/filters/?collection_id=qin_prostate_repeatability)
*   Goal : Prostate segmentation using Task005 Prostate nnU-net pre-trained model, T2 and ADC multi-modal input

# Auth login

##Colab

In [ ]:
#colab 
from google.colab import auth
auth.authenticate_user()

## Drive setup for upload results

In [ ]:
# import os
# #drive mount
# from google.colab import drive
# drive.mount('/content/drive')
# ##setup paths
# path_drive_root = "/content/drive/MyDrive/"
# drive_exp_root_name = "IDC" 
# data = "qin-prostate-repeatability"
# pre_trained_mod = "task005-Prostate"
# timestamp = "2022_07_12_ADC_2D"#year_month_day_MOD

In [ ]:
# #personal drive path -- make sure folder exists in google drive
# path_experiment = os.path.join(path_drive_root,
#                                drive_exp_root_name,
#                                data,
#                                pre_trained_mod,
#                                timestamp)

# if not os.path.exists(path_experiment):
#   !mkdir -p $path_experiment

# Global variables

## Variables used for resampling -- inference -- setup labelID for ground truth segs

In [ ]:
# IDC collection for paths setup
collection_name = "qin_prostate_root_data" #up to user
##nnunet
model_type = '3d_fullres'#other options are '2d',..
nnunet_labelID = 1 #for task005 model, labelid=1 means PZ and 2 means TZ
##labelID seg retrieval
ground_truth_cat_CodeMeaning = 'Anatomical Structure'
ground_truth_type_CodeMeaning = 'Peripheral zone of the prostate'
#Resampling scheme -- this option is made for qin-rep-repeatability(SeriesDescription tag dependence,etc), not modular for other collections with multi modal inference input
desired_grid = "T2" #Other option is "ADC" 

## Global paths

In [ ]:
import os 

#qin_prostate_repeatibility
os.environ["qin_prostate_rep_root"] = os.path.join(os.getcwd(), collection_name)
os.environ["qin_prostate_rep_dicom"] = os.path.join(os.environ["qin_prostate_rep_root"], "dicom")
os.environ["qin_prostate_rep_nii"] = os.path.join(os.environ["qin_prostate_rep_root"], "nii")
os.environ["qin_prostate_rep_nrrd"] = os.path.join(os.environ["qin_prostate_rep_root"], "nrrd")
os.environ["qin_prostate_rep_analysis"] = os.path.join(os.environ["qin_prostate_rep_root"], "results")
os.environ["qin_prostate_rep_analysis_data"] = os.path.join(os.environ["qin_prostate_rep_analysis"], "data_seg")
os.environ["qin_prostate_rep_analysis_verbose"] = os.path.join(os.environ["qin_prostate_rep_root"], "results_verbose")


#resampled data -- To t2 mod
os.environ["qin_prostate_rep_root_resampled"] = os.path.join(os.environ["qin_prostate_rep_root"], "resampled")
os.environ["qin_prostate_rep_nii_resampled"] = os.path.join(os.environ["qin_prostate_rep_root_resampled"], "nii")
os.environ["qin_prostate_rep_nrrd_resampled"] = os.path.join(os.environ["qin_prostate_rep_root_resampled"], "nrrd")

#nnunet
os.environ["nnUNet"] = os.path.join(os.getcwd(), "nnUNet")
os.environ["nnUNet_data"] = os.path.join(os.environ["nnUNet"], "data")
os.environ['nnUNet_raw_data_base'] = os.path.join(os.environ["nnUNet_data"], "nnUNet_raw_data")
os.environ['nnUNet_preprocessed'] = os.path.join(os.environ["nnUNet_data"], "processed")
os.environ["nnUNet_models"] = os.path.join(os.environ["nnUNet"], "models")
os.environ["RESULTS_FOLDER"] = os.path.join(os.environ["nnUNet"], "output", "preds")
os.environ["nnUNet_preds_resampled"] = os.path.join(os.environ["nnUNet"], "output", "preds_resampled")
os.environ["nnUNet_raw_data"] = os.path.join(os.environ["nnUNet"], "raw_data")
#path where model pre-trained weights are stored
os.environ["PATH_TO_MODEL_FILE"] = os.path.join(os.environ["nnUNet"], "models", "Task005_Prostate.zip")

#misc
os.environ["IDC_Downloads"] = os.path.join(os.getcwd(), "IDC_DL")
os.environ["IDC_Downloads_Sorted"] = os.path.join(os.getcwd(), "IDC_DL", "Sorted")
os.environ["logs"] = os.path.join(os.getcwd(), "logs")

#create dirs for specific folders names
for key, path in os.environ.items():
  check_patterns = [True for el in ["qin", "nnunet", "IDC", "nnUNet", "logs"] if el in key]
  if True in check_patterns:
    !mkdir -p $path

# Setup GCP Project ID

In [ ]:
project_id = "idc-nlst-unet-seg"
os.environ["GCP_PROJECT_ID"] = project_id

# Download nnUnet pre-trained model on prostate decathlon data -- task005 -- personal dropbox

nnUnet pre-trained models zip files can also be found [here](https://zenodo.org/record/4003545#.YsWmH-zMLt8).

In [ ]:
print(os.environ["nnUNet_models"])

/content/nnUNet/models


In [ ]:
# this will usually take between one and five minutes (but can sometimes take up to eight)
seg_model_url = "https://www.dropbox.com/s/igpwt45v6hlquxp/Task005_Prostate.zip?dl=0"
out_path_mod = os.path.join(os.environ["nnUNet_models"], "Task005_Prostate.zip")#"/content/nnUnet/models/Task024_Promise.zip"
!wget -O $out_path_mod $seg_model_url

--2022-07-14 18:24:40--  https://www.dropbox.com/s/igpwt45v6hlquxp/Task005_Prostate.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/igpwt45v6hlquxp/Task005_Prostate.zip [following]
--2022-07-14 18:24:40--  https://www.dropbox.com/s/raw/igpwt45v6hlquxp/Task005_Prostate.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc33a6697211f09dde130cbb2eda.dl.dropboxusercontent.com/cd/0/inline/BpHNGZs1K1LCkQUuPRSjS1CqMyfxyVXA6m2Z1JWAqLO9bWKjUpWmEVmDprb0eFZcPFvRJtTkvIWx3LgGGMIrA2dSwnULJDZgAGiCwfhmSf6G64aSA_YHKA2FGtszQOCAWbBP2NEeeMA1xvDAopXXoN_i8eMwV7Mp-YzKcI1aqCgS2Q/file# [following]
--2022-07-14 18:24:41--  https://uc33a6697211f09dde130cbb2eda.dl.dropboxusercontent.com/cd/0/inline/BpHNGZs1K1LCkQUuPRSjS1CqMyfxyVXA6m2Z1JW

# Setup of the Colab VM



In the following cells we will confirm you have a GPU before doing anything else, and will install and import all the Python dependencies. 

The main python packages we need to install are:
* `nnunet` - which is the [codebase for the nn-UNet framework](https://github.com/MIC-DKFZ/nnUNet) we are going to be using for the segmentation step;
* `pydicom`, a Python [package](https://github.com/pydicom/pydicom) that lets the use read, modify, and write DICOM data in an easy "pythonic" way - that we are going to use to distinguish different DICOM objects from each other.

## GPU checks

In [ ]:
# check wether the Colab Instance was correctly initialized with a GPU instance
gpu_list = !nvidia-smi --list-gpus

has_gpu = False if "failed" in gpu_list[0] else True

if not has_gpu:
  print("Your Colab VM does not have a GPU - check \"Runtime > Change runtime type\"")

In [ ]:
# check which model of GPU the notebook is equipped with - a Tesla K80 or T4
# T4 is the best performing on the two - and can about half the GPU processing time

!nvidia-smi

Thu Jul 14 18:25:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Environment Setup

Here we will configure the Linux environment variables needed to run the nnU-Net pipeline. 

Three main variables are needed by default to run the nnU-Net segmentation pipelines:
* `nnUNet_raw_data_base` is the path to the folder where the segmentation pipeline expects to find the data to process;
* `nnUNet_preprocessed` is the path to the folder where the preprocessed data are saved;
* `RESULTS_FOLDER` is the path to the folder storing by default the model weights and, in our case, for simplicity, the segmentation masks produced by the pipeline.

We will use the additional variable `PATH_TO_MODEL_FILE` to point to the location where the pre-trained model weights for the chosen model will be stored (more on this later).

Please notice that these variables need to be set using `os.environ[]` in Google Colab - as `!export` is not sufficient to guarantee the variables are kept from one cell to the other. For more in-depth information regarding what the nnU-Net framework uses these folders for, please visit [the dedicated nnU-Net documentation page](https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/setting_up_paths.md)

## Install command-line tools


[Plastimatch](https://plastimatch.org/index.html) is considered to be the swiss army knife of medical images processing: we will use it to convert DICOM (CT, RTSTRUCT) series to NRRD files - but it can be used for a multitude of other tasks, such as registration, resampling, cropping, and computing statistics to name a few. Plastimatch is also available as a 3DSlicer plug-in and can be used directly from the Slicer GUI.

For the sake of clarity and simplicity, we will call Plastimatch from a very simple [Python wrapper](https://github.com/denbonte/pyplastimatch) written for the occasion (unfortunately, Plastimatch does not provide an official one).

In [ ]:
%%capture
!sudo apt update

!sudo apt install plastimatch

In [ ]:
!echo $(plastimatch --version)

plastimatch version 1.7.0


[dcmqi](https://github.com/QIICR/dcmqi) is an open source library that can help with the conversion between imaging research formats and the standard DICOM representation for image analysis results. More specifically, you can use dcmqi convert DICOM Segmentation objects (DICOM SEG) into research formats, such as NIfTI and NRRD.

In [ ]:
%%capture
!wget https://github.com/QIICR/dcmqi/releases/download/v1.2.4/dcmqi-1.2.4-linux.tar.gz
!tar zxvf dcmqi-1.2.4-linux.tar.gz
!cp dcmqi-1.2.4-linux/bin/* /usr/local/bin/

Finally, we are going to install [Subversion](https://subversion.apache.org/), a tool that will allow us to clone GitHub repositories only partially (to save time and space).

In [ ]:
%%capture

!sudo apt install subversion

In [ ]:
!echo $(svn --version | head -n 2)

svn, version 1.9.7 (r1800392) compiled May 21 2022, 07:24:25 on x86_64-pc-linux-gnu


## Install Python packages

In [ ]:
%%capture
!pip install nnunet
!pip install pydicom
!pip install nibabel

Unpack and install model we downloaded earlier (under `PATH_TO_MODEL_FILE`). This step can take about 1-2 minutes.

In [ ]:
%%capture
!nnUNet_install_pretrained_model_from_zip $PATH_TO_MODEL_FILE

Next we set up few things to help with visualization of the segmentations later.

In [ ]:
import os
import sys
import shutil
import csv
import random

import os
import glob
import csv
import json

import nibabel as nib

import time
import gdown

import json
import pprint
import numpy as np
import pandas as pd

import pydicom
import nibabel as nib
import SimpleITK as sitk

from medpy.metric.binary import dc as dice_coef
from medpy.metric.binary import hd as hausdorff_distance
from medpy.metric.binary import asd as avg_surf_distance

from medpy.filter.binary import largest_connected_component

# use the "tensorflow_version" magic to make sure TF 1.x is imported
%tensorflow_version 1.x
import tensorflow as tf
import keras

print("Python version               : ", sys.version.split('\n')[0])
print("Numpy version                : ", np.__version__)
print("TensorFlow version           : ", tf.__version__)
print("Keras (stand-alone) version  : ", keras.__version__)

print("\nThis Colab instance is equipped with a GPU.")

After that, `%tensorflow_version 1.x` will throw an error.

Your notebook should be updated to use Tensorflow 2.
See the guide at https://www.tensorflow.org/guide/migrate#migrate-from-tensorflow-1x-to-tensorflow-2.

TensorFlow 1.x selected.
Python version               :  3.7.13 (default, Apr 24 2022, 01:04:09) 
Numpy version                :  1.21.6
TensorFlow version           :  1.15.2
Keras (stand-alone) version  :  2.3.1

This Colab instance is equipped with a GPU.


Using TensorFlow backend.


In [ ]:
# PyPlastimatch - python wrapper for Plastimatch (and interactive notebook visualisation)
!svn checkout https://github.com/AIM-Harvard/pyplastimatch/trunk/pyplastimatch pyplastimatch

A    pyplastimatch/__init__.py
A    pyplastimatch/pyplastimatch.py
A    pyplastimatch/utils
A    pyplastimatch/utils/__init__.py
A    pyplastimatch/utils/data.py
A    pyplastimatch/utils/eval.py
A    pyplastimatch/utils/viz.py
Checked out revision 18.


In [ ]:
# dicomsort is the pythong package that can sort DICOM files into
# folder organization based on user-specified DICOM attributes
!git clone https://github.com/pieper/dicomsort.git

Cloning into 'dicomsort'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 130 (delta 0), reused 1 (delta 0), pack-reused 126
Receiving objects: 100% (130/130), 44.12 KiB | 11.03 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [ ]:
from pyplastimatch import pyplastimatch as pypla
from pyplastimatch.utils import viz as viz_utils
from pyplastimatch.utils import data as data_utils

# Data selection, downloading and structuring

We want to select here the collection named qin-prostate repeatibility, and more particularly the two timepoints per patient ID for further analysis.

In order to use data hosted by IDC effectively, you will need to utilize metadata to navigate what data is available and to select specific files that are relevant in your analysis. The main metadata table you will need for this purpose is the [`bigquery-public-data.idc_current.dicom_all`](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=idc_current&t=dicom_all&page=table) table.

This query has one row per file hosted by IDC. All of IDC data is in DICOM format, and each of the rows in this table will have all of the DICOM attributes extracted from a given file. It will also have various columns containing non-DICOM metadata, such as the name of the collection where the file is included, size of the file, and URL that can be used to retrieve that file.

To query IDC BigQuery tables, you can use one of the following approaches:
1. `%%bigquery` magic will allow you to define your query in plain SQL, and load the result of the query into a Pandas dataframe.
2. [BigQuery Python API](https://googleapis.dev/python/bigquery/latest/index.html) is more flexible in allowing you to parameterize your query.
3. [Google Cloud BigQuery console](https://console.cloud.google.com/bigquery) is very convenient for interactive query exploration of tables.
4. [`gcloud bq`](https://cloud.google.com/bigquery/docs/bq-command-line-tool) is the command line tool that comes as part of [Cloud SDK](https://cloud.google.com/sdk) and is convenient for scripting interactions from the shell. Cloud SDK is preinstalled on Colab.

In the following cells we will utilize `%%bigquery`, Python BigQuery SDK and BigQuery console for working with IDC BigQuery tables.

First, to verify that you are authenticated, and your project ID is working, let's run a test query against IDC BigQuery table to get the summary statistics about the  of data available in IDC.


Given `SeriesInstanceUID` value identifying the image series, we can query the IDC metadata table to get the list of files (defined by the Google Storage URLs) corresponding to this series.

All of the DICOM metadata for each of the DICOM files is available in the BigQuery table we will be querying. We will get not just the `gcs_url`, but also identifiers for the Study, Series and Instance, to better understand organization of data, and since `StudyInstanceUID` will be handy later when we get to the visualization of the data.

## Data selection -- BigQuery

Explanation

In [ ]:
from google.cloud import bigquery
bq_client = bigquery.Client(os.environ["GCP_PROJECT_ID"])
selection_query = f"""
  SELECT  
  StudyInstanceUID, 
  SeriesInstanceUID, 
  SOPInstanceUID, 
  PatientID, 
  Modality, 
  collection_id, 
  StudyDescription, 
  SeriesDescription, 
  StudyDate, 
  SeriesDate,
  gcs_url,
  (SELECT SeriesInstanceUID FROM UNNEST(ReferencedSeriesSequence)) AS RefSerieUID 
FROM 
  `bigquery-public-data.idc_current.dicom_all` 
WHERE 
  collection_id = 'qin_prostate_repeatability' 
  AND SeriesDescription IN ('T2 Weighted Axial', 'Apparent Diffusion Coefficient', 
    'T2 Weighted Axial Segmentations', 'Apparent Diffusion Coefficient Segmentations')""" 

In [ ]:
selection_result = bq_client.query(selection_query)
selection_df = selection_result.result().to_dataframe()

## Download gs urls into dcm files and sorting  

## Randomly pick X PatientIDs for the use case - pick 5 by default

In [ ]:
# # # #random PatientID selection
# # patientID_select = random.sample(list(selection_df.PatientID.values), 5)#X randomly picked PatientIDs 

# # static PatientID selection
# patientID_select = ["PCAMPMRI-00010"]

# selection_df = selection_df[selection_df.PatientID.isin(patientID_select)]

In [ ]:
selection_df.PatientID.unique()

array(['PCAMPMRI-00010'], dtype=object)

In [ ]:
# save the list of GCS URLs into a file
selection_manifest = os.path.join(os.environ["IDC_Downloads"], "idc_manifest.txt")
selection_df["gcs_url"].to_csv(selection_manifest, header=False, index=False)

In [ ]:
# confirm the resulting manifest has as many lines as the number of rows in the
# dataframe we initialized earlier
!cat {selection_manifest} |wc

    112     112    7392


In [ ]:
# let's make sure the download folder is clean, in case you ran this cell earlier
# for a different dataset
!rm -rf {os.environ["IDC_Downloads"]+"/*.dcm"}

In [ ]:
# download is this simple!
%%capture

!cat {selection_manifest} | gsutil -m cp -I {os.environ["IDC_Downloads"]}

To facilitate preparation of the data, we will sort the downloaded files into the PatientID/StudyInstanceUID/SeriesInstanceUID hierarchy.

In [ ]:
!python dicomsort/dicomsort.py -k -u {os.environ["IDC_Downloads"]} {os.environ["IDC_Downloads_Sorted"]}/%PatientID/%StudyInstanceUID/%SeriesInstanceUID/%SOPInstanceUID.dcm

100% 113/113 [00:00<00:00, 160.34it/s]
Files sorted


Move the sorted data into the right place

In [ ]:
rm -rf {os.environ["qin_prostate_rep_dicom"]+"/*"}* && mv {os.environ["IDC_Downloads_Sorted"]+"/*"} {os.environ["qin_prostate_rep_dicom"]}

#Pre-Processing

Dicom files are assumed to be sorted this way for further processing :

* PatientID
  * StudyUID
    * SeriesUID
      * *.dcm file(s)

## conversion methods

https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=24283641

switch from plastimatch resample method to simple itk for slice spacing issue

In [ ]:
def safe_sitk_read(img_list, *args, **kwargs):
  """
  Since the default function just looks at images 0 and 1 to determine slice thickness
  and the images are often not correctly alphabetically sorted, much slower
  :param img_list:
  :return:
  """
  pimg_list = [(sitk.ReadImage(x).GetOrigin(), x) for x in img_list]
  s_img_list = [path for _, path in sorted(pimg_list, key = lambda x: x[0][2])] # sort by z
  return sitk.ReadImage(s_img_list, *args, **kwargs)

In [ ]:
def convert_mr_to_nii_nrrd(input_path, output_path_root, modality, target_format="nii.gz"):
  if not os.path.exists(output_path_root): 
    !mkdir -p $output_path_root
  output_path = os.path.join(output_path_root, input_path.split('/')[-1][:-4] 
                             + "_MR_" + modality
                             + "." + target_format)
  #convert to target format
  if not os.path.exists(output_path):
    dcm_img = safe_sitk_read(glob.glob(os.path.join(input_path, "**", "*.dcm"), recursive=True))
    sitk.WriteImage(dcm_img, output_path)

We will use the [segimage2itkimage tool](https://qiicr.gitbook.io/dcmqi-guide/opening/cmd_tools/seg/segimage2itkimage) from dcmqi to extract individual segments from DICOM SEG series we just downloaded into NII format, saving each segment into a separate file

In [ ]:
def convert_seg_to_nii(input_path, output_path):
  if not os.path.exists(output_path): 
    !mkdir -p $output_path
  
  print(f'input path : {input_path}')
  print(f'output_path : {output_path}')
  !segimage2itkimage --inputDICOM $input_path --outputDirectory $output_path \
  --outputType nii 

In [ ]:
def convert_dcm_sorted(input_path,output_path):
  path_dicom = os.environ["qin_prostate_rep_dicom"] 
  for serie_folder in sorted(glob.glob(os.path.join(input_path, "**", "**", "*"))):#, recursive = True):
    path_serie_dcm_lst = glob.glob(os.path.join(serie_folder, "*.dcm"))
    modality = selection_df[selection_df["SeriesInstanceUID"] == path_serie_dcm_lst[0].split('/')[-2]]["Modality"].iloc[0]#'SEG' if pydicom.dcmread(path_serie_dcm_lst[0]).Modality == "SEG" else "MR"
    seriesInstanceUID = serie_folder.split("/")[-1]
    studyInstanceUID = serie_folder.split("/")[-2]
    patientID = serie_folder.split("/")[-3]
    SeriesDescription = selection_df[selection_df["SeriesInstanceUID"] == path_serie_dcm_lst[0].split('/')[-2]]["SeriesDescription"].iloc[0]
    type_serie = ["T2" if "T2" in SeriesDescription else "ADC"][0]
    if modality == "SEG":
      #convert to nrrd
      convert_seg_to_nii(input_path=path_serie_dcm_lst[0],
                              output_path=os.path.join(output_path, "nii", 
                                                       patientID, studyInstanceUID, seriesInstanceUID + "_{}_".format(type_serie) +"SEG"))
    elif modality == "MR":
      #convert to nii
      convert_mr_to_nii_nrrd(input_path=serie_folder,
                              output_path_root=os.path.join(output_path, "nii", 
                                                       patientID, studyInstanceUID, seriesInstanceUID + "_MR"), 
                             modality=type_serie, target_format="nii.gz")
      #convert to nrrd
      convert_mr_to_nii_nrrd(input_path=serie_folder, 
                              output_path_root=os.path.join(output_path, "nrrd", 
                                                       patientID, studyInstanceUID, seriesInstanceUID + "_MR"), 
                             modality=type_serie, target_format="nrrd")
    else:
      print(f"type serie {modality} not supported")

## Conversion to NII/NRRD

In [ ]:
!rm -rf /content/qin_prostate_root_data/nii/*
!rm -rf /content/qin_prostate_root_data/nrrd/*

In [ ]:
convert_dcm_sorted(input_path=os.environ["qin_prostate_rep_dicom"],
                   output_path=os.environ["qin_prostate_rep_root"])

input path : /content/qin_prostate_root_data/dicom/PCAMPMRI-00010/1.3.6.1.4.1.14519.5.2.1.3671.4754.217251077299095981018478951589/1.2.276.0.7230010.3.1.3.1426846371.18344.1513205168.159/1.2.276.0.7230010.3.1.4.1426846371.18344.1513205168.160.dcm
output_path : /content/qin_prostate_root_data/nii/PCAMPMRI-00010/1.3.6.1.4.1.14519.5.2.1.3671.4754.217251077299095981018478951589/1.2.276.0.7230010.3.1.3.1426846371.18344.1513205168.159_ADC_SEG
dcmqi repository URL: git@github.com:QIICR/dcmqi.git revision: ef9e227 tag: v1.2.4
W: CodeValue (0008,0100) empty in CodeSequenceMacro (type 1C)
Row direction: 0.999063 0.0432741 -0.000180629
Col direction: -0.0427421 0.987428 0.15218
Z direction: 0.00676382 -0.15203 0.988353
Total frames: 96
Total frames with unique IPP: 24
Total overlapping frames: 24
Origin: [-113.348, -60.7389, -24.2662]
input path : /content/qin_prostate_root_data/dicom/PCAMPMRI-00010/1.3.6.1.4.1.14519.5.2.1.3671.4754.217251077299095981018478951589/1.2.276.0.7230010.3.1.3.142684637

# Resampling ground truth segmentations to desired grid

In [ ]:
!rm -rf /content/qin_prostate_root_data/resampled/nii/*
!rm -rf /content/qin_prostate_root_data/resampled/nrrd/*

In [ ]:
nii_lst = glob.glob(os.path.join(os.environ["qin_prostate_rep_nii"], "**", "*_SEG", "*.nii.gz"), recursive=True)
for seg_nii in sorted(nii_lst) :
  print(seg_nii)
  patientID = seg_nii.split('/')[-4]
  studyUID = seg_nii.split('/')[-3]
  serieUID = seg_nii.split('/')[-2].split('_')[0] #standard serieUID without T2_SEG custom suffix
  type_serie = '_'.join(seg_nii.split('/')[-2].split('_')[1:])
  #get referencedSerieUID from seg corresponding to T2 MR volume 
  referencedSerieUID = str(selection_df[selection_df["SeriesInstanceUID"] == serieUID]["RefSerieUID"].iloc[0])	
  matched_mod = glob.glob(os.path.join(os.environ['qin_prostate_rep_nii'], 
                                            patientID, studyUID, referencedSerieUID + "_MR", "*.nii*"))[0]
  output_path_seg_root = os.path.join(os.environ["qin_prostate_rep_nii_resampled"],
                                  patientID, studyUID,
                                  serieUID + f"_{type_serie}")
  if not os.path.exists(output_path_seg_root):
    !mkdir -p $output_path_seg_root
  resample_args = {"input" : seg_nii, 
                        "output" : os.path.join(output_path_seg_root, seg_nii.split('/')[-1]),
                        "fixed" : matched_mod,
                        "interpolation" : "nn"}
  #resample to t2 mod based on study -- not uniform across studies
  pypla.resample(verbose = False, **resample_args)

/content/qin_prostate_root_data/nii/PCAMPMRI-00010/1.3.6.1.4.1.14519.5.2.1.3671.4754.217251077299095981018478951589/1.2.276.0.7230010.3.1.3.1426846371.18344.1513205168.159_ADC_SEG/1.nii.gz
/content/qin_prostate_root_data/nii/PCAMPMRI-00010/1.3.6.1.4.1.14519.5.2.1.3671.4754.217251077299095981018478951589/1.2.276.0.7230010.3.1.3.1426846371.18344.1513205168.159_ADC_SEG/2.nii.gz
/content/qin_prostate_root_data/nii/PCAMPMRI-00010/1.3.6.1.4.1.14519.5.2.1.3671.4754.217251077299095981018478951589/1.2.276.0.7230010.3.1.3.1426846371.18344.1513205168.159_ADC_SEG/3.nii.gz
/content/qin_prostate_root_data/nii/PCAMPMRI-00010/1.3.6.1.4.1.14519.5.2.1.3671.4754.217251077299095981018478951589/1.2.276.0.7230010.3.1.3.1426846371.18344.1513205168.159_ADC_SEG/4.nii.gz
/content/qin_prostate_root_data/nii/PCAMPMRI-00010/1.3.6.1.4.1.14519.5.2.1.3671.4754.217251077299095981018478951589/1.2.276.0.7230010.3.1.3.1426846371.4124.1513205167.5_T2_SEG/1.nii.gz
/content/qin_prostate_root_data/nii/PCAMPMRI-00010/1.3.6.1.

# Resampling input data(T2--ADC) to desired T2 or ADC resampling grid

In [ ]:
nii_lst = glob.glob(os.path.join(os.environ["qin_prostate_rep_nii"], "**", "*_MR", "*.nii.gz"), recursive=True)
for mr_nii in sorted(nii_lst) :
  # print(mr_nii)
  patientID = mr_nii.split('/')[-4]
  studyUID = mr_nii.split('/')[-3]
  serieUID = mr_nii.split('/')[-2].split('_')[0] #standard serieUID without T2_SEG custom suffix
  sopInstanceUID = mr_nii.split('/')[-1]
  type_serie = mr_nii.split('/')[-1].split('_')[-1].split('.')[0]
  # print(f'type_serie : {type_serie}')
  # print(f'serieUID : {serieUID}')
  output_path_root = os.path.join(os.environ["qin_prostate_rep_nii_resampled"],
                                  patientID, studyUID, mr_nii.split('/')[-2]) 
  output_path = os.path.join(output_path_root,sopInstanceUID) 
  if not os.path.exists(output_path_root):
    !mkdir -p $output_path_root
  if desired_grid != type_serie:
    match_des_grid = glob.glob(os.path.join(os.environ['qin_prostate_rep_nii'],
                                         patientID, studyUID, 
                                         "*_MR*", #serieUID
                                         f"*{desired_grid}*.nii*"))[0] #SOPUID
    # print(f"match_des_grid : {match_des_grid}")
    resample_args = {"input" : mr_nii, 
                        "output" : output_path,
                        "fixed" : match_des_grid}
    #resample based on study -- not uniform across studies
    pypla.resample(verbose = False, **resample_args)
  elif desired_grid == type_serie :#just cp other volume for now, no fixed resampling grid option implemented yet
    !cp $mr_nii $output_path
  else:
    pass

# Inference -- nnUnet

## Naming Formatting files for nnUnet inference

In [ ]:
t2_list = glob.glob(os.path.join(os.environ["qin_prostate_rep_nii_resampled"], 
                                 "**", "**", "*_MR*", f"*T2.nii.gz"), recursive=True)
adc_list = glob.glob(os.path.join(os.environ["qin_prostate_rep_nii_resampled"], 
                                 "**", "**", "*_MR*", f"*ADC.nii.gz"), recursive=True)
matchStudyUID = lambda x: (x[0], x[1]) if x[0].split('/')[-3] == x[1].split('/')[-3] else None
sorted_mr_lst = [matchStudyUID((t2_mr, adc_mr)) 
                    for t2_mr in t2_list 
                        for adc_mr in adc_list 
                            if matchStudyUID((t2_mr, adc_mr)) is not None]
#make sure studyUIDs match
for el in sorted_mr_lst : assert el[0].split('/')[-3] == el[1].split('/')[-3]
#name formatting and copying to target folder
for mr_t2_adc in sorted_mr_lst:
  for mr_vol in mr_t2_adc: 
    studyInstanceUID = mr_vol.split('/')[-3]
    patientID = mr_vol.split("/")[-4]
    nnunet_idx = "0000" if "T2" in mr_vol.split('/')[-1] else "0001"#0000 for T2 and 0001 for ADC
    nnunet_path = os.path.join(os.environ["nnUNet_preprocessed"], 
                                "_".join([patientID, studyInstanceUID, nnunet_idx]) + ".nii.gz") 
    !cp $mr_vol $nnunet_path

## Inference

In [ ]:
!nnUNet_print_pretrained_model_info Task005_Prostate



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

Prostate Segmentation. 
Segmentation targets are peripheral and central zone, 
Input modalities are 0: T2, 1: ADC. 
Also see Medical Segmentation Decathlon, http://medicaldecathlon.com/


In [ ]:
print(f"input folder path : "+os.environ["nnUNet_preprocessed"])
print(f"output folder path : "+os.environ["RESULTS_FOLDER"])

input folder path : /content/nnUNet/data/processed
output folder path : /content/nnUNet/output/preds


In [ ]:
!echo {os.environ["nnUNet_preprocessed"]} 

/content/nnUNet/data/processed


In [ ]:
%%capture
!nnUNet_install_pretrained_model_from_zip $out_path_mod

In [ ]:
!nnUNet_predict --input_folder {os.environ["nnUNet_preprocessed"]} \
                --output_folder {os.environ["RESULTS_FOLDER"]} \
                --task_name "Task005_Prostate" --model $model_type \
                --save_npz \



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

using model stored in  /content/nnUNet/output/preds/nnUNet/3d_fullres/Task005_Prostate/nnUNetTrainerV2__nnUNetPlansv2.1
This model expects 2 input modalities for each image
Found 2 unique case ids, here are some examples: ['PCAMPMRI-00010_1.3.6.1.4.1.14519.5.2.1.3671.4754.318227959935022390930294557376'
 'PCAMPMRI-00010_1.3.6.1.4.1.14519.5.2.1.3671.4754.318227959935022390930294557376']
If they don't look right, make sure to double check your filenames. They must end with _0000.nii.gz etc
number of cases: 2
number of cases that still need to be predicted: 2
emptying cuda cache
loading parameters for folds, None
folds is None so we 

# Resamplings preds back to input data -- desired grid

In [ ]:
for pred_path in sorted(glob.glob(os.path.join(os.environ["RESULTS_FOLDER"], "*.nii.gz"))):
  search_t2_path = os.path.join(os.environ["qin_prostate_rep_nii_resampled"], pred_path.split('/')[-1].split('_')[0], #get PatientID
                                   pred_path.split('/')[-1].split('_')[-1][:-7], #get studyUID
                                    "**", #SerieUID
                                   f"*{desired_grid}.nii.gz") #SOPUID
  print(f"search path for {desired_grid} : {search_t2_path}")
  t2_path = glob.glob(search_t2_path, recursive=True)[0]                              
  print(f"{desired_grid}_path found : {t2_path}")
  print(f"pred path : {pred_path}")
  resample_args_to_t2_origin = {"input" : pred_path, 
                        "output" : os.path.join(os.environ["nnUNet_preds_resampled"], 
                                                pred_path.split('/')[-1]),
                        "fixed" : t2_path,
                        "interpolation" : "nn"}
  
  path_log = os.path.join(os.environ["logs"], 'log_pypla_res_pred' + pred_path.split('/')[-1].split('.')[0] + '.txt')            
  !touch $path_log
  pypla.resample(verbose = False, **resample_args_to_t2_origin, path_to_log_file=path_log)
  print()

search path for T2 : /content/qin_prostate_root_data/resampled/nii/PCAMPMRI-00010/1.3.6.1.4.1.14519.5.2.1.3671.4754.217251077299095981018478951589/**/*T2.nii.gz
T2_path found : /content/qin_prostate_root_data/resampled/nii/PCAMPMRI-00010/1.3.6.1.4.1.14519.5.2.1.3671.4754.217251077299095981018478951589/1.3.6.1.4.1.14519.5.2.1.3671.4754.218938411300600171613807256501_MR/1.3.6.1.4.1.14519.5.2.1.3671.4754.21893841130060017161380725_MR_T2.nii.gz
pred path : /content/nnUNet/output/preds/PCAMPMRI-00010_1.3.6.1.4.1.14519.5.2.1.3671.4754.217251077299095981018478951589.nii.gz

search path for T2 : /content/qin_prostate_root_data/resampled/nii/PCAMPMRI-00010/1.3.6.1.4.1.14519.5.2.1.3671.4754.318227959935022390930294557376/**/*T2.nii.gz
T2_path found : /content/qin_prostate_root_data/resampled/nii/PCAMPMRI-00010/1.3.6.1.4.1.14519.5.2.1.3671.4754.318227959935022390930294557376/1.3.6.1.4.1.14519.5.2.1.3671.4754.105976129314091491952445656147_MR/1.3.6.1.4.1.14519.5.2.1.3671.4754.105976129314091491952

# Analysis

## Functions aiming to retrieve right ground truth labelsID per nnUnet prediction

In [ ]:
def retrieve_seg_labelID(input_path_json, categoryCode = 'Anatomical Structure', typeCode ='Prostate'):
  #read json file
  json_dic = json.load(open(input_path_json))
  for seg in json_dic['segmentAttributes']:
    cat_code = seg[0].get('SegmentedPropertyCategoryCodeSequence').get('CodeMeaning')
    type_code = seg[0].get('SegmentedPropertyTypeCodeSequence').get('CodeMeaning')
    current_labelID = seg[0].get('labelID')
    if cat_code.lower().find(categoryCode.lower()) != -1 and type_code.lower().find(typeCode.lower()) != -1:
      return current_labelID#return first occurence of match
  return 'None'

In [ ]:
def get_seg_per_mod(input_path_lst, multi_mod='', categoryCode='Anatomical Structure', typeCode='Peripheral zone of the prostate'):#multi_mod == either T2 or ADC
  res_lst = []
  for nii_file in input_path_lst:
    nii_file_patientID = nii_file.split('/')[-1].split('_')[0]
    nii_file_studyInstanceUID = nii_file.split('/')[-1].split('_')[-1][:-7]
    if multi_mod == '' : 
      serieUID_seg = selection_df[(selection_df.PatientID == nii_file_patientID) \
                                  & (selection_df.StudyInstanceUID == nii_file_studyInstanceUID) \
                                  & (selection_df.Modality == "SEG")]["SeriesInstanceUID"].iloc[0]
    elif multi_mod == "T2":
      serieUID_seg = selection_df[(selection_df.PatientID == nii_file_patientID) \
                                  & (selection_df.StudyInstanceUID == nii_file_studyInstanceUID) \
                                  & (selection_df.Modality == "SEG") \
                                  & (selection_df.SeriesDescription.str.contains(multi_mod))]["SeriesInstanceUID"].iloc[0]
    elif multi_mod == "ADC":
      serieUID_seg = selection_df[(selection_df.PatientID == nii_file_patientID) \
                                  & (selection_df.StudyInstanceUID == nii_file_studyInstanceUID) \
                                  & (selection_df.Modality == "SEG") \
                                  & (selection_df.SeriesDescription.str.contains("Apparent Diffusion Coefficient"))]["SeriesInstanceUID"].iloc[0]#to implement -- seriesDescription here is Apparent ... and not ADC
    else:
      pass
    json_path = glob.glob(os.path.join(os.environ["qin_prostate_rep_nii"], 
                                       nii_file_patientID, 
                                       nii_file_studyInstanceUID, 
                                       serieUID_seg+f"_{multi_mod}"+"_SEG", "*.json"))[0] 
    labelID = retrieve_seg_labelID(json_path, 
                                   categoryCode=categoryCode, 
                                   typeCode=typeCode)                              
    res_lst.append(os.path.join(os.environ["qin_prostate_rep_nii_resampled"], 
                                nii_file_patientID, 
                                nii_file_studyInstanceUID, 
                                serieUID_seg+f"_{multi_mod}"+"_SEG", str(labelID)+".nii.gz"))
  return res_lst

## Dice score computation 

In [ ]:
!rm -rf {os.environ["qin_prostate_rep_analysis_verbose"]}#analysis_verbose/
!mkdir -p {os.environ["qin_prostate_rep_analysis_verbose"]}

### Compute dice score between preds and ground truth and store the results into analysis folder -- for T2 serie

In [ ]:
# !rm -rf qin_prostate_root_data/results/data_seg/*

In [ ]:
def convert_mask_binary(input_path, output_path_root, labelID=1):
  assert os.path.isfile(input_path)
  output_path = os.path.join(output_path_root, input_path.split('/')[-1])
  if not os.path.exists(output_path_root):
    !mkdir -p $output_path_root
  img_input = nib.load(input_path)
  img_output = nib.load(input_path).get_data().copy()
  # print(f"img input header : {img_input.header}")
  img_output = np.where(img_output == labelID, 1, 0)
  nib.save(nib.nifti1.Nifti1Image(img_output, 
                                  img_input.affine, 
                                  header=img_input.header),
                                  output_path)
  return output_path

In [ ]:
ref_path_lst = get_seg_per_mod(sorted(glob.glob(os.path.join(os.environ['nnUNet_preds_resampled'], 
                                                             "**", "*.nii.gz"), 
                                                recursive=True)),
                               categoryCode=ground_truth_cat_CodeMeaning, 
                               typeCode=ground_truth_type_CodeMeaning, 
                               multi_mod=desired_grid)
for ref_path in sorted(ref_path_lst):
  print(f"seg path  {ref_path}")
  ref_path_PatientID = ref_path.split('/')[-4]
  ref_path_StudyInstanceUID = ref_path.split('/')[-3]
  pred_path = glob.glob(os.path.join(os.environ["nnUNet_preds_resampled"], 
                                     '_'.join([ref_path_PatientID, ref_path_StudyInstanceUID]) + '*.nii.gz'))[0]
  output_path = os.environ["qin_prostate_rep_analysis_verbose"]
  #convert ground truth segs to binary 
  ref_path = convert_mask_binary(ref_path, 
                      output_path_root=os.path.join(os.environ["qin_prostate_rep_analysis_data"],
                                                    ref_path_PatientID,
                                                    ref_path_StudyInstanceUID), 
                      labelID=int(ref_path.split('/')[-1][:-7]))
  #convert pred segs to binary 
  pred_path = convert_mask_binary(pred_path, 
                      output_path_root=os.path.join(os.environ["qin_prostate_rep_analysis_data"],
                                                    pred_path.split('/')[-1].split('_')[0],
                                                    pred_path.split('/')[-1].split('_')[-1][:-7]), 
                      labelID=nnunet_labelID)#==2 for task05 supposed to mean TZ label and 1 == PZ
  print(f"pred path : {pred_path}")
  print(f"ref path : {ref_path}")
  print()
  if not os.path.exists(output_path):
    !mkdir -p $output_path 
  log_path = pred_path.split('/')[-1][:-7]+"_dice_score_verbose.txt"
  !echo "ref_path {ref_path}" > {os.path.join(output_path,log_path)}
  !echo "pred_path {pred_path}" >> {os.path.join(output_path,log_path)}
  !plastimatch dice --all $ref_path $pred_path >> {os.path.join(output_path,log_path)}

seg path  /content/qin_prostate_root_data/resampled/nii/PCAMPMRI-00010/1.3.6.1.4.1.14519.5.2.1.3671.4754.217251077299095981018478951589/1.2.276.0.7230010.3.1.3.1426846371.4124.1513205167.5_T2_SEG/2.nii.gz


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  import sys


pred path : /content/qin_prostate_root_data/results/data_seg/PCAMPMRI-00010/1.3.6.1.4.1.14519.5.2.1.3671.4754.217251077299095981018478951589/PCAMPMRI-00010_1.3.6.1.4.1.14519.5.2.1.3671.4754.217251077299095981018478951589.nii.gz
ref path : /content/qin_prostate_root_data/results/data_seg/PCAMPMRI-00010/1.3.6.1.4.1.14519.5.2.1.3671.4754.217251077299095981018478951589/2.nii.gz

seg path  /content/qin_prostate_root_data/resampled/nii/PCAMPMRI-00010/1.3.6.1.4.1.14519.5.2.1.3671.4754.318227959935022390930294557376/1.2.276.0.7230010.3.1.3.1426846371.7872.1513205171.131_T2_SEG/2.nii.gz
pred path : /content/qin_prostate_root_data/results/data_seg/PCAMPMRI-00010/1.3.6.1.4.1.14519.5.2.1.3671.4754.318227959935022390930294557376/PCAMPMRI-00010_1.3.6.1.4.1.14519.5.2.1.3671.4754.318227959935022390930294557376.nii.gz
ref path : /content/qin_prostate_root_data/results/data_seg/PCAMPMRI-00010/1.3.6.1.4.1.14519.5.2.1.3671.4754.318227959935022390930294557376/2.nii.gz



### store dice score into dictionary

In [ ]:
dice_dic = {}
#iterate txt logs for dice_score computation
for txt_file in sorted(glob.glob(os.path.join(os.environ["qin_prostate_rep_analysis_verbose"], "*.txt"))): 
  with open(txt_file, encoding='utf8') as f:
    #variable placeholders for txt file info -- temporary
    pholder_dice = ''
    pholder_center_mass_pred = ''
    pholder_center_mass_ref = ''
    pholder_hsdrff = ''
    pholder_ref = ''
    pholder_test = ''
    for line in f:
        if "DICE" in line:
          pholder_dice = float(line.strip().split()[-1])
        elif "ref" in line and len(line.strip().split()) > 2:
          pholder_center_mass_ref = ' '.join([line.strip().split()[-3], 
                                             line.strip().split()[-2], line.strip().split()[-1]])
        elif "cmp" in line and len(line.strip().split()) > 2:
          pholder_center_mass_pred = ' '.join([line.strip().split()[-3], 
                                             line.strip().split()[-2], line.strip().split()[-1]])                                             
        elif len(line.strip().split()) <= 5 and 'Hausdorff distance' in line:
          pholder_hsdrff = float(line.strip().split()[-1])
        elif "ref_path" in line:
          pholder_ref = str(line.strip().split()[-1]) 
        elif "pred_path" in line:
          pholder_test = str(line.strip().split()[-1])
        else:
          pass
    patientID = txt_file.split('/')[-1][:-4].split('_')[0] #retrieve studyInstanceUID from txt file name 
    study = txt_file.split('/')[-1][:-4].split('_')[-4] #retrieve patientID
    # push values to dict
    dice_dic[study] = {'PatientID' : patientID,
                       'dice_score' : pholder_dice, 
                       'center_mass_pred' : pholder_center_mass_pred,
                       'center_mass_ref' : pholder_center_mass_ref,
                       'hausdorff_dist' : pholder_hsdrff,
                       'ground_truth_seg_path' : pholder_ref,
                       'pred_nnnet_path' : pholder_test}            

# print('-----------------------')
# print(f"average dice score : {np.average(np.array([val['dice_score'] \
                                                  #  for val in dice_dic.values()]))}")
# print('-----------------------')
# print()
for key,val in dice_dic.items():
  study = key
  patientID = val["PatientID"]
  dice_score = val["dice_score"]
  cm_pred = val["center_mass_pred"]
  cm_ref = val["center_mass_ref"]
  hausdorff_dist = val["hausdorff_dist"]
  gt_path = val["ground_truth_seg_path"]
  pred_path = val["pred_nnnet_path"]
  print(f"study : {study}")
  print(f"patientID : {patientID}")
  print(f"ground truth seg path : {gt_path}")
  print(f"pred seg masks nnunet path : {pred_path}")
  print(f"dice_score : {dice_score}")
  print(f"center mass ref : {cm_ref}")
  print(f"center mass pred : {cm_pred}")
  print(f"hausdorff_dist : {hausdorff_dist}")
  print()


study : 1.3.6.1.4.1.14519.5.2.1.3671.4754.217251077299095981018478951589
patientID : PCAMPMRI-00010
ground truth seg path : /content/qin_prostate_root_data/results/data_seg/PCAMPMRI-00010/1.3.6.1.4.1.14519.5.2.1.3671.4754.217251077299095981018478951589/2.nii.gz
pred seg masks nnunet path : /content/qin_prostate_root_data/results/data_seg/PCAMPMRI-00010/1.3.6.1.4.1.14519.5.2.1.3671.4754.217251077299095981018478951589/PCAMPMRI-00010_1.3.6.1.4.1.14519.5.2.1.3671.4754.217251077299095981018478951589.nii.gz
dice_score : 0.30375
center mass ref : -29.6323 26.4219 26.6483
center mass pred : -38.9709 19.9104 28.6678
hausdorff_dist : 21.738583

study : 1.3.6.1.4.1.14519.5.2.1.3671.4754.318227959935022390930294557376
patientID : PCAMPMRI-00010
ground truth seg path : /content/qin_prostate_root_data/results/data_seg/PCAMPMRI-00010/1.3.6.1.4.1.14519.5.2.1.3671.4754.318227959935022390930294557376/2.nii.gz
pred seg masks nnunet path : /content/qin_prostate_root_data/results/data_seg/PCAMPMRI-00010/1.

In [ ]:
#Save dice_score to csv
with open(os.path.join(os.environ['qin_prostate_rep_analysis'], 'analysis_metrics.csv'), 'w') as f:
    f.write("StudyInstanceUID,PatientID,dice_score,center_mass_ref,center_mass_pred,hausdorff_dist, ground_truth_seg_path, pred_nnnet_path\n")
    for key,val in dice_dic.items():
      study = str(key)
      patientID = str(val["PatientID"])
      dice_score = float(val["dice_score"])
      cm_pred = str(val["center_mass_pred"])
      cm_ref = str(val["center_mass_ref"])
      hausdorff_dist = float(val["hausdorff_dist"])
      gt_path = str(val["ground_truth_seg_path"])
      pred_path = str(val["pred_nnnet_path"])
      f.write(f"{study},{patientID},{dice_score},{cm_ref},{cm_pred},{hausdorff_dist},{gt_path},{pred_path}\n")

### All metrics

In [ ]:
ref_path_lst = get_seg_per_mod(sorted(glob.glob(os.path.join(os.environ['nnUNet_preds_resampled'],"**", "*.nii.gz"), 
                                                recursive=True)), multi_mod="T2")
for ref_path in sorted(ref_path_lst):
  ref_path_PatientID = ref_path.split('/')[-4]
  ref_path_StudyInstanceUID = ref_path.split('/')[-3]
  test_path = glob.glob(os.path.join(os.environ["nnUNet_preds_resampled"], 
                                     '_'.join([ref_path_PatientID, ref_path_StudyInstanceUID]) + '.nii.gz'))[0]
  output_path = "analysis/txt/"
  if not os.path.exists(output_path):
    !mkdir -p $output_path 
  log_path = test_path.split('/')[-1][:-7]+".txt"
  print(f"ref path : {ref_path}")
  print(f"test path : {test_path}")
  !plastimatch dice --all $ref_path $test_path 
  print()
  print()

ref path : /content/qin_prostate_root_data/resampled/nii/PCAMPMRI-00010/1.3.6.1.4.1.14519.5.2.1.3671.4754.217251077299095981018478951589/1.2.276.0.7230010.3.1.3.1426846371.4124.1513205167.5_T2_SEG/2.nii.gz
test path : /content/nnUNet/output/preds_resampled/PCAMPMRI-00010_1.3.6.1.4.1.14519.5.2.1.3671.4754.217251077299095981018478951589.nii.gz
CENTER_OF_MASS
ref	      -29.6323	       26.4219	       26.6483
cmp	      -19.6602	       6.73351	       34.1053
TP:         11665
TN:       7079308
FN:         32398
FP:        216661
DICE:      0.085650
SE:      0.264735
SP:      0.970304
Hausdorff distance = 60.943108
Avg average Hausdorff distance = 10.772633
Max average Hausdorff distance = 17.691158
Percent (0.95) Hausdorff distance = 31.372925
Hausdorff distance (boundary) = 60.943108
Avg average Hausdorff distance (boundary) = 12.795958
Max average Hausdorff distance (boundary) = 21.316694
Percent (0.95) Hausdorff distance (boundary) = 33.015324


ref path : /content/qin_prostate_root_data/

# Export results to google drive 

## define input and output paths for idc_data and nnunet in google drive through colab

In [ ]:
# #path to IDC data
# input_idc_data = os.environ["qin_prostate_rep_root"]
# output_drive_idc_data = os.path.join(path_experiment, "idc_data")
# if not os.path.exists(output_drive_idc_data):
#   !mkdir -p $output_drive_idc_data
  
# #path to nnunet preds
# input_nnunet = os.environ["nnUNet"]
# output_nnunet = os.path.join(path_experiment, "nnUNet")
# if not os.path.exists(output_nnunet):
#   !mkdir -p $output_nnunet

In [ ]:
# #sanity checks regarding storage size
# ##data
# !du -sh $input_idc_data
# ##preds
# !du -sh $input_nnunet

## Copy data to drive

In [ ]:
# #copy idc data
# %cp -av $input_idc_data $output_drive_idc_data
# #verbose True, keep same struct as input
# #copy nnunet data
# %cp -av $input_nnunet $output_nnunet